In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.applications import InceptionV3

# Step 1: Load and Prepare the Dataset
def load_dataset(data_dir):
    images, labels = [], []
    for label in os.listdir(data_dir):
        label_dir = os.path.join(data_dir, label)
        if os.path.isdir(label_dir):
            for img_name in os.listdir(label_dir):
                img_path = os.path.join(label_dir, img_name)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (160, 160))  # Resize for InceptionV3
                images.append(img)
                labels.append(label)
    return np.array(images), np.array(labels)

# Step 2: Encode Faces using FaceNet
def create_embedding_model():
    base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(160, 160, 3))
    base_model.trainable = False  # Freeze the base model
    x = Flatten()(base_model.output)
    x = Dense(512, activation='relu')(x)
    model = Model(inputs=base_model.input, outputs=x)
    return model

# Step 3: Train the Model
def train_model(images, labels):
    model = create_embedding_model()
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Encode images
    embeddings = model.predict(images)

    # Convert labels to one-hot encoding
    from sklearn.preprocessing import LabelBinarizer
    lb = LabelBinarizer()
    labels_encoded = lb.fit_transform(labels)

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(embeddings, labels_encoded, test_size=0.2, random_state=42)

    # Train a classifier (e.g., SVM)
    from sklearn.svm import SVC
    classifier = SVC(kernel='linear', probability=True)
    classifier.fit(X_train, y_train)

    # Evaluate the model
    y_pred = classifier.predict(X_test)
    print(classification_report(y_test, y_pred, target_names=lb.classes_))
    return model, classifier, lb

# Step 4: Recognize Faces
def recognize_face(model, classifier, lb, img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (160, 160))
    img = np.expand_dims(img, axis=0)

    embedding = model.predict(img)
    prediction = classifier.predict(embedding)
    predicted_label = lb.inverse_transform(prediction)

    plt.imshow(cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB))
    plt.title(f"Predicted: {predicted_label[0]}")
    plt.axis('off')
    plt.show()

# Step 5: Main Function
if __name__ == "__main__":
    data_directory = 'path_to_lfw_dataset'  # Update this path to your LFW dataset
    images, labels = load_dataset(data_directory)
    print(f"Loaded {len(images)} images.")

    model, classifier, lb = train_model(images, labels)

    # Test the system on a new image
    test_image_path = 'path_to_test_image.jpg'  # Update this path to your test image
    recognize_face(model, classifier, lb, test_image_path)